In [1]:
import uuid
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, JSON, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID

Base = declarative_base()

# Association tables for many-to-many relationships
requirement_function_association = Table(
    'requirement_function', Base.metadata,
    Column('requirement_id', Integer, ForeignKey('requirements.id')),
    Column('function_id', Integer, ForeignKey('functions.id'))
)

function_physical_association = Table(
    'function_physical', Base.metadata,
    Column('function_id', Integer, ForeignKey('functions.id')),
    Column('physical_id', Integer, ForeignKey('physicals.id'))
)

# Self-referential association tables for sub-requirements, sub-functions, and sub-physicals
sub_requirement_association = Table(
    'sub_requirement', Base.metadata,
    Column('parent_id', Integer, ForeignKey('requirements.id')),
    Column('child_id', Integer, ForeignKey('requirements.id'))
)

sub_function_association = Table(
    'sub_function', Base.metadata,
    Column('parent_id', Integer, ForeignKey('functions.id')),
    Column('child_id', Integer, ForeignKey('functions.id'))
)

sub_physical_association = Table(
    'sub_physical', Base.metadata,
    Column('parent_id', Integer, ForeignKey('physicals.id')),
    Column('child_id', Integer, ForeignKey('physicals.id'))
)

# Define the top-level Component model with UUID
class Component(Base):
    __tablename__ = 'components'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    name = Column(String, nullable=False)
    description = Column(String, nullable=True)

    # One-to-many relationship with requirements
    requirements = relationship("Requirement", back_populates="component")

    def __repr__(self):
        return f"<Component(id={self.id}, uuid={self.uuid}, name='{self.name}')>"

# Define the Requirement model with UUID and JSON data
class Requirement(Base):
    __tablename__ = 'requirements'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    component_id = Column(Integer, ForeignKey('components.id'), nullable=True)  # Made nullable for sub-requirements
    data = Column(JSON, nullable=False)  # Store Requirement as JSON

    component = relationship("Component", back_populates="requirements")
    functions = relationship("Function", secondary=requirement_function_association, back_populates="requirements")
    
    # Self-referential relationship for sub-requirements
    sub_requirements = relationship(
        "Requirement", secondary=sub_requirement_association,
        primaryjoin=id == sub_requirement_association.c.parent_id,
        secondaryjoin=id == sub_requirement_association.c.child_id,
        backref="parent_requirements"
    )

    def __repr__(self):
        return f"<Requirement(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Function model with UUID and JSON data
class Function(Base):
    __tablename__ = 'functions'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Function as JSON

    requirements = relationship("Requirement", secondary=requirement_function_association, back_populates="functions")
    physicals = relationship("Physical", secondary=function_physical_association, back_populates="functions")
    
    # Self-referential relationship for sub-functions
    sub_functions = relationship(
        "Function", secondary=sub_function_association,
        primaryjoin=id == sub_function_association.c.parent_id,
        secondaryjoin=id == sub_function_association.c.child_id,
        backref="parent_functions"
    )

    def __repr__(self):
        return f"<Function(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Physical model with UUID and JSON data
class Physical(Base):
    __tablename__ = 'physicals'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Physical as JSON

    functions = relationship("Function", secondary=function_physical_association, back_populates="physicals")
    
    # Self-referential relationship for sub-physical elements
    sub_physicals = relationship(
        "Physical", secondary=sub_physical_association,
        primaryjoin=id == sub_physical_association.c.parent_id,
        secondaryjoin=id == sub_physical_association.c.child_id,
        backref="parent_physicals"
    )

    def __repr__(self):
        return f"<Physical(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Database setup
engine = create_engine('sqlite:///db_model_4.db')
Base.metadata.create_all(engine)

# Session setup
Session = sessionmaker(bind=engine)
session = Session()


Step 1: Insert Requirements into the Database
The data field in your model can store the JSON object directly. Here’s how to add the provided data:

In [ ]:
# import uuid
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy import create_engine
# from models import Requirement, Base  # Assuming Requirement is defined in models.py

# # Step 1: Set up the database engine and session
# engine = create_engine("sqlite:///example.db")  # Replace with your database URL
# Session = sessionmaker(bind=engine)
# session = Session()

# Step 2: Prepare the data
data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop."
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door."
    },
    # Add the rest of your data here...
]

# Step 3: Insert data into the database
for item in data:
    requirement = Requirement(
        uuid=uuid.uuid4(),  # Generate a new UUID
        data=item  # Store the entire JSON object
    )
    session.add(requirement)

# Step 4: Commit the transaction
session.commit()

print("Requirements added successfully!")


Step 2: Verify the Data
To check that the requirements were added correctly:

In [ ]:
# Query all requirements
requirements = session.query(Requirement).all()

for req in requirements:
    print(f"ID: {req.id}, UUID: {req.uuid}, Data: {req.data}")


separate json for the req and funciton data

In [ ]:


# Step 2: Prepare the data (Separate JSON for requirements and functions)
requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop."
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door."
    },
    # Add more requirements here...
]

functions_data = [
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    },
    # Add more functions here...
]

# Step 3: Insert Requirements and Functions, linking by `index`
for req_data in requirements_data:
    # Insert Requirement
    requirement = Requirement(
        uuid=uuid.uuid4(),  # Generate a new UUID
        data={"index": req_data["index"], "title": req_data["title"], "description": req_data["description"]}
    )
    session.add(requirement)
    session.commit()  # Commit to get the `id` of the requirement

    # Insert linked Functions
    for func_data in functions_data:
        if func_data["requirement_index"] == req_data["index"]:
            # Insert Function only if it matches the requirement index
            function = Function(
                data={"description": func_data["description"], "operation": func_data["operation"]}
            )
            session.add(function)
            session.commit()  # Commit to get the `id` of the function

            # Associate Function with Requirement
            requirement.functions.append(function)
            session.commit()

# Step 4: Commit the transaction
session.commit()

print("Requirements and linked functions added successfully!")


2. insert retrieve and then insert funcitons

In [2]:
import uuid
from sqlalchemy.orm import joinedload

# Step 2: Prepare the data (Separate JSON for requirements and functions)
requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop."
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door."
    },
    # Add more requirements here...
]

functions_data = [
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    },
    # Add more functions here...
]



# Step 1: Insert Requirements into the database
for req_data in requirements_data:
    # Create a new Requirement object
    requirement = Requirement(
        uuid=uuid.uuid4(),  # Generate a new UUID
        data={
            "index": req_data["index"],
            "title": req_data["title"],
            "description": req_data["description"],
        },
    )
    # Add the requirement to the session
    session.add(requirement)

# Commit all requirements to the database
session.commit()
print("Requirements inserted successfully!")

# Step 2: Retrieve the inserted requirements
retrieved_requirements = session.query(Requirement).all()

# Create a dictionary to map requirement indices to Requirement objects
requirement_map = {req.data["index"]: req for req in retrieved_requirements}

# Step 3: Insert Functions and associate with Requirements
for func_data in functions_data:
    # Get the Requirement object by its index
    requirement = requirement_map.get(func_data["requirement_index"])

    if requirement:
        # Create a new Function object
        function = Function(
            data={
                "description": func_data["description"],
                "operation": func_data["operation"],
            }
        )
        # Add the function to the session
        session.add(function)
        session.commit()  # Commit to get the function ID

        # Associate the function with the retrieved requirement
        requirement.functions.append(function)

# Final commit to save the associations
session.commit()
print("Functions inserted and associated with requirements successfully!")


Requirements inserted successfully!
Functions inserted and associated with requirements successfully!


Alternative: Linking by id (after inserting Requirements)
If you want to link by id (after the requirements are inserted), you can modify the code to store the id of each requirement after insertion and then link the functions by id.

Here’s how you would adjust the code to link by id:

In [ ]:
import uuid

requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop."
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door."
    },
    # Add more requirements here...
]

functions_data = [
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    },
    # Add more functions here...
]


# Step 3: Insert Requirements and Functions, linking by `uuid`
requirement_map = {}

# Insert Requirements and store them in a map by their UUID
for req_data in requirements_data:
    # Generate a new UUID for the requirement
    requirement_uuid = uuid.uuid4()

    # Insert Requirement with the generated UUID in both `uuid` and `data`
    requirement = Requirement(
        uuid=requirement_uuid,  # Use the generated UUID
        data={
            "index": req_data["index"],
            "title": req_data["title"],
            "description": req_data["description"],
            "requirement_uuid": str(requirement_uuid)  # Add the UUID to the `data` field as well
        }
    )
    session.add(requirement)
    session.commit()  # Commit to get the `id` of the requirement

    # Store the requirement in the map with its UUID as the key
    requirement_map[str(requirement_uuid)] = requirement

# Now insert the functions and link by `uuid`
for func_data in functions_data:
    # Insert Function with the associated requirement's UUID
    function = Function(
        data={
            "description": func_data["description"],
            "operation": func_data["operation"],
            "requirement_uuid": func_data["requirement_index"]  # Use the UUID to link the function
        }
    )
    session.add(function)
    session.commit()  # Commit to get the `id` of the function

    # Link Function to Requirement by `uuid`
    requirement_uuid = func_data["requirement_index"]  # Get the UUID from the function data
    requirement = requirement_map.get(requirement_uuid)  # Look up the requirement by UUID
    if requirement:
        requirement.functions.append(function)
        session.commit()

# Step 4: Commit the transaction
session.commit()

print("Requirements and linked functions added successfully!")



In [ ]:
# uuid based linking

In [ ]:

requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop."
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door."
    },
    # Add more requirements here...
]

functions_data = [
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    },
    # Add more functions here...
]




# Step 3: Insert Requirements and Functions, linking by `id`
requirement_map = {}

for req_data in requirements_data:
    # Insert Requirement
    # Generate a new UUID for the requirement
    requirement_uuid = uuid.uuid4()

    # Insert Requirement with the generated UUID in both `uuid` and `data`
    requirement = Requirement(
        uuid=requirement_uuid,  # Use the generated UUID
        data={
            "index": req_data["index"],
            "title": req_data["title"],
            "description": req_data["description"],
            "requirement_uuid": str(requirement_uuid)  # Add the UUID to the `data` field as well
        }
    )
    session.add(requirement)
    session.commit()  # Commit to get the `id` of the requirement

    # Store the requirement in the map with its `index` as the key
    requirement_map[req_data["index"]] = requirement

# Now insert the functions and link by `id`
for func_data in functions_data:
    # Insert Function
    function = Function(
        data={"description": func_data["description"], "operation": func_data["operation"]}
    )
    session.add(function)
    session.commit()  # Commit to get the `id` of the function

    # Link Function to Requirement by `id`
    requirement = requirement_map.get(func_data["requirement_index"])
    if requirement:
        requirement.functions.append(function)
        session.commit()

# Step 4: Commit the transaction
session.commit()

print("Requirements and linked functions added successfully!")


requirement_index based linking 

In [ ]:

requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop.",
        "uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304"
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door.",
        "uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93"
    }
]

functions_data = [
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_index": "1",  # Links to the requirement with index 1
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_index": "2",  # Links to the requirement with index 2
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    },
    # Add more functions here...
]# Create functions based on the requirements
functions_data = [
    {
        "index": "1",
        "title": "Automatic Reversing",
        "description": "The system should automatically reverse the garage door operation when an obstacle is detected.",
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",
        "uuid": str(uuid.uuid4())
    },
    {
        "index": "2",
        "title": "Safety Sensors Activation",
        "description": "The system should activate safety sensors to detect obstructions and prevent accidents.",
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",
        "uuid": str(uuid.uuid4())
    },
    {
        "index": "3",
        "title": "Emergency Stop Mechanism",
        "description": "The system should provide a manual or automatic emergency stop feature to halt the garage door in case of failure.",
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",
        "uuid": str(uuid.uuid4())
    },
    {
        "index": "4",
        "title": "Remote Control Operation",
        "description": "Users should be able to operate the garage door opener remotely using a provided remote control device.",
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",
        "uuid": str(uuid.uuid4())
    },
    {
        "index": "5",
        "title": "Wall Console Functionality",
        "description": "Users should be able to open and close the garage door via a wall-mounted console.",
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",
        "uuid": str(uuid.uuid4())
    },
    {
        "index": "6",
        "title": "Smartphone App Integration",
        "description": "The system should integrate with a smartphone app to allow door monitoring and operation.",
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",
        "uuid": str(uuid.uuid4())
    }
]



# Step 3: Insert Requirements and Functions, linking by `id`
requirement_map = {}

for req_data in requirements_data:
    # Insert Requirement
    # Generate a new UUID for the requirement
    requirement_uuid = uuid.uuid4()

    # Insert Requirement with the generated UUID in both `uuid` and `data`
    requirement = Requirement(
        uuid=requirement_uuid,  # Use the generated UUID
        data={
            "index": req_data["index"],
            "title": req_data["title"],
            "description": req_data["description"],
            "requirement_uuid": str(requirement_uuid)  # Add the UUID to the `data` field as well
        }
    )
    session.add(requirement)
    session.commit()  # Commit to get the `id` of the requirement

    # Store the requirement in the map with its `index` as the key
    requirement_map[req_data["index"]] = requirement

# Now insert the functions and link by `id`
for func_data in functions_data:
    # Insert Function
    function = Function(
        data={"description": func_data["description"], "operation": func_data["operation"]}
    )
    session.add(function)
    session.commit()  # Commit to get the `id` of the function

    # Link Function to Requirement by `id`
    requirement = requirement_map.get(func_data["requirement_index"])
    if requirement:
        requirement.functions.append(function)
        session.commit()

# Step 4: Commit the transaction
session.commit()

print("Requirements and linked functions added successfully!")


query the association

Simple Query to Get Functions for a Requirement:
You can query a Requirement and retrieve all the associated Function objects through the relationship.

In [ ]:
# Query for a specific requirement (by index or id)
requirement = session.query(Requirement).filter_by(id = 1).first()

# Query for a specific requirement by UUID
# requirement = session.query(Requirement).filter(Requirement.uuid == UUID("4e91294ccbfa471aa6bd4f99e8d933fc")).first()

# Access the associated functions
if requirement:
    print(f"Requirement: {requirement.data['title']}")
    for function in requirement.functions:
        print(f"Function: {function.data['description']}, Operation: {function.data['operation']}")
else:
    print("Requirement not found.")


In [3]:
from sqlalchemy.orm import joinedload

# Query requirements with functions using a join
requirements_with_functions = session.query(Requirement).options(joinedload(Requirement.functions)).all()

for requirement in requirements_with_functions:
    print(f"Requirement: {requirement.data['title']}")
    for function in requirement.functions:
        print(f"  - Function: {function.data['description']}, Operation: {function.data['operation']}")


Requirement: Safety Features
  - Function: Detect obstacles, Operation: Use sensors to detect objects in the path of the door.
  - Function: Emergency stop, Operation: Immediately halt the door movement when an obstacle is detected.
Requirement: User Interface
  - Function: Remote control operation, Operation: Allow users to operate the door remotely.
  - Function: Smartphone app integration, Operation: Provide a mobile app for controlling and monitoring the door.
